<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Magíster en Data Science</em><br>
    <em>Profesor: Tomás Fontecilla </em><br>

</div>

# Machine Learning Avanzado
*02 de Diciembre de 2024*

#### Integrantes: 
` Gabriel Álvarez - Rosario Valderrama `

## 1. Objetivo

El objetivo de este informe es ajustar **redes neuronales convolucionales** para realizar un análisis de ciencia de datos utilizando la base extraída de Kaggle ` chihuahuas vs muffins `. Luego, compararemos los resultados con un **modelo de perceptrón multicapa**.

A ambos modelos se aplicará 3 estructuras diferentes: 
- Regularización L2
- Dropout
- Decay

In [136]:
#!pip install setuptools

In [137]:
#!pip install -r requirements.txt

## 2. Introducción

A través del dataset ` chihuahuas vs muffins ` que son imágenes, se busca entrenar un modelo de clasificación de redes neuronales (perceptrón multicapa y convolucionales) para lograr clasificar correctamente cada imagen. Para esto, los pasos a seguir son:

1. Cargar las imágenes
2. Preprocesar las imágenes
3. Crear el modelo
4. Entrenar el modelo
5. Evaluar el modelo
6. Guardar el modelo
7. Hacer predicciones

## 3. Metodología

### 3.1 Perceptrón Multicapa (MLP)

Para comenzar, se carga las imágenes desde las carpetas data/train y data/test, donde se infieren las etiquetas de ellas basándose en los nombres de las subcarpetas. Todas las imágenes se redimensionan a 128 x 128 para asegurar que todas tengan el mismo tamaño, para luego agruparlas en lotes de 32 imágenes, lo que permite poder procesarlas de forma más eficiente. Todas las imágenes son "barajadas" del conjunto de entrenamiento, para así evitar patrones en el orden de los datos que finalmente afecte este entrenamiento.

Se normalizan los valores de cada pixel de las imágenes, dividiéndolos por 255, para que así estén dentro del rango de 0 a 1. Si no se aplicara esta normalización, los valores podrían estar en el rango del 0 a 255, dificultando así la convergencia del modelo.

In [143]:
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers, models, Sequential

# Directorios
train_dir = "data/train"
test_dir = "data/test"

# 1. Cargar las imágenes
train_dataset = image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),  # Tamaño fijo
    batch_size=32,
    shuffle=True
)

test_dataset = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    shuffle=False
)

# Inspección de los datos
class_names = train_dataset.class_names
print(f"Clases detectadas: {class_names}")

# Normalización (valores entre 0 y 1)
normalization_layer = layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

Found 4733 files belonging to 2 classes.
Found 1184 files belonging to 2 classes.
Clases detectadas: ['chihuahua', 'muffin']


Dado a que el perceptrón multicapa no puede procesar las imágenes en 2D, requiere que las imágenes se transformen en vectores 1D. Para esto, se busca aplanar las imágenes, convirtiendo cada imagen de 128 x 128 x 3 (RGB) en un vector 1D de tamaño 49152.

In [145]:
# Aplanar las imágenes para el MLP
# Convierte imágenes 2D en vectores 1D
def flatten_images(x, y):
    flat_dim = 128 * 128 * 3  # Tamaño fijo de las imágenes aplanadas
    x = tf.reshape(x, [tf.shape(x)[0], flat_dim])  # Garantiza que sea un tensor válido
    return x, y

train_dataset_flat = train_dataset.map(flatten_images)
test_dataset_flat = test_dataset.map(flatten_images)


El modelo del perceptrón multicapa se construye utilizando la API **Sequiential** de TensorFlow/Keras, lo que permite definir una arquitectura de red neuronal de manera secuencial, que va atravesando capa por capa.

La primera capa, llamada Input, especifica la forma de las entradas del modelo, que corresponde a vectores del tamaño 128 x 128 x 3 = 49152, resultado de aplanar las imágenes RGB de entrada que previamente se redimencionaron a 128 x 128 pixeles. 

Posteriormente, se definen 3 capas ocultas densas llamadas **dense**, que están completamente conectadas. Esto significa que cada neurona de una capa está conectada a todas las neuronas de la siguiente capa. La primera capa oculta contiene 256 neuronas, la segunda 128 neuronas y la tercera 64 neuronas.

Todas las capas ocultas usan la función de activación ReLU,que se selecciona por su capacidad para introducir no linealidad en la red, permitiendo que el modelo aprenda relaciones complejas entre los datos de entrada y los de salida. Esta función activa únicamente valores positivos (los deja sin cambios) y transforma los valores negativos a cero. Esto mejora la eficiencia computacional en comparación con otras funciones de activación y ayuda a evitar problemas como el desvanecimiento del gradiente, que puede ralentizar el aprendizaje en redes profundas. Al activar solo ciertas neuronas según las entradas, ReLU también ayuda a que la red se enfoque en características relevantes, optimizando su capacidad de generalización.

Finalmente, la capa de salida también es densa y tiene tantas neuronas como clases en el problema, lo cual se especifica como len(class_names). En este caso, las clases corresponden a las categorías de las imágenes (chihuahuas y muffins). Esta capa utiliza la función de activación softmax, que convierte las salidas en probabilidades normalizadas, asignando una probabilidad a cada clase. Se utiliza esta activación ya que permite interpretar la salida del modelo como la probabilidad de que una imagen pertenezca a cada clase.

En conjunto, estas capas y funciones de activación permiten que el modelo procese los datos de entrada, aprenda patrones significativos en las imágenes y genere predicciones probabilísticas sobre las clases a las que pertenece cada imagen. 

In [147]:
# Crear el modelo MLP
mlp_model = Sequential([
    layers.Input(shape=(128*128*3,)),  # Imágenes aplanadas (128x128x3)
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')  # Número de clases
])

**mlp_model.compile** configura el modelo de Perceptrón Multicapa para el entrenamiento, definiendo tres componentes clave: el optimizador, la función de pérdida y las métricas de evaluación. El optimizador Adam se utiliza por su capacidad para ajustar dinámicamente la tasa de aprendizaje, combinando las ventajas de métodos como Momentum y RMSProp. La función de pérdida sparse_categorical_crossentropy es adecuada para problemas de clasificación multiclase con etiquetas enteras, como las clases inferidas de las imágenes. Finalmente, se especifica la métrica de exactitud (accuracy), que mide la proporción de predicciones correctas durante el entrenamiento y la evaluación, proporcionando una evaluación clara del desempeño del modelo.

In [149]:
# Compilar el modelo
mlp_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
mlp_model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_47 (Dense)                     │ (None, 256)                 │      12,583,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_48 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_49 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_50 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,624,450 (48.16 MB)

 Trainable params: 12,624,450 (48.16 MB)

 Non-trainable params: 0 (0.00 B)

**mlp_model.fit** se utiliza para entrenar el modelo con los datos del conjunto de entrenamiento (train_dataset_flat). La cantidad de "epochs" se define mediante el parámetro epochs=10, indicando que el modelo realizará 10 iteraciones completas sobre todo el conjunto de datos de entrenamiento. Durante cada "epoch", el modelo ajusta sus pesos en función de los gradientes calculados para minimizar la función de pérdida. Cada pasada completa permite al modelo mejorar su capacidad para aprender los patrones en los datos, mientras que un número excesivo de "epochs" podría llevar al sobreajuste, donde el modelo se adapta demasiado a los datos de entrenamiento y pierde capacidad de generalización para datos nuevos. 

In [151]:
# Entrenar el modelo
mlp_model.fit(train_dataset_flat, epochs=10)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 16s 105ms/step - accuracy: 0.5374 - loss: 4.9561
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 16s 106ms/step - accuracy: 0.6151 - loss: 1.0347
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 16s 107ms/step - accuracy: 0.6731 - loss: 0.6810
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 16s 107ms/step - accuracy: 0.7103 - loss: 0.6112
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 16s 107ms/step - accuracy: 0.7008 - loss: 0.6054
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 16s 107ms/step - accuracy: 0.7366 - loss: 0.5580
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 16s 108ms/step - accuracy: 0.7549 - loss: 0.5134
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 16s 107ms/step - accuracy: 0.7545 - loss: 0.5127
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 16s 107ms/step - accuracy: 0.7197 - loss: 0.5790
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 16s 107ms/step - accuracy: 0.7734 - loss: 0.4930


**mlp_model.evaluate** permite evaluar el desempeño del modelo en el conjunto de datos de prueba (test_dataset_flat). Este proceso calcula la pérdida, que mide qué tan bien o mal se comporta el modelo con datos nuevos, y el accuracy, que indica el porcentaje de predicciones correctas. 

La pérdida ayuda a diagnosticar posibles problemas en el ajuste del modelo, mientras que el accuracy refleja su capacidad de generalización y su desempeño al momento de clasificar las imágenes de salida. 

In [132]:
# Evaluar el modelo
loss, accuracy = mlp_model.evaluate(test_dataset_flat)
print(f"Pérdida del MLP: {loss}")
print(f"Exactitud del MLP: {accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.8584 - loss: 0.9732
Pérdida del MLP: 0.9919908046722412
Exactitud del MLP: 0.5405405163764954


In [97]:
from tensorflow.keras import regularizers

# Crear el modelo MLP con regularización
mlp_model = Sequential([
    layers.Input(shape=(128*128*3,)),  # Imágenes aplanadas (128x128x3)
    
    # Primera capa densa con regularización L2 y Dropout
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Dropout del 30%

    # Segunda capa densa con regularización L2 y Dropout
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Dropout del 30%

    # Tercera capa densa con regularización L2
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),

    # Capa de salida (número de clases)
    layers.Dense(len(class_names), activation='softmax')  # len(class_names) = número de clases
])

# Compilar el modelo
mlp_model.compile(
    optimizer='adam',  # Puedes añadir decay si lo deseas
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
mlp_model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_36 (Dense)                     │ (None, 256)                 │      12,583,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_37 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_38 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_39 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,624,450 (48.16 MB)

 Trainable params: 12,624,450 (48.16 MB)

 Non-trainable params: 0 (0.00 B)

#### 3.1.1 Perceptrón Multicapa, aplicando técnicas de regularización para prevenir el sobreajuste.

Para prevenir el sobreajuste en este modelo, se recomienda utilizar una combinación de tres técnicas de regularización: Dropout, Regularización L2, y Decay. 

**Dropout** introduce robustez al modelo al apagar aleatoriamente un porcentaje de las neuronas durante el entrenamiento, obligando al modelo a aprender patrones más generalizados en lugar de depender de combinaciones específicas de pesos. Esto se implementa añadiendo capas de Dropout con un porcentaje, como layers.Dropout(0.3) para desactivar el 30% de las neuronas en cada capa. 

**Regularización L2**, penaliza los pesos grandes al añadir una restricción basada en la suma de los cuadrados de los pesos, lo que fomenta que el modelo mantenga distribuciones de pesos más balanceadas. Se configura con el argumento kernel_regularizer=regularizers.l2(0.01) en las capas densas.

**Decay** ajusta dinámicamente la tasa de aprendizaje del optimizador, permitiendo pasos grandes al inicio para aprender rápidamente y pasos más pequeños al final para refinar los pesos; esto se implementa mediante un programa de decaimiento como ExponentialDecay. 

Estas técnicas ayudan a mejorar la generalización del modelo al reducir la complejidad, distribuir el aprendizaje de manera uniforme y evitar la dependencia excesiva en subconjuntos específicos de parámetros.

In [100]:
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Decaimiento de la tasa de aprendizaje
learning_rate_schedule = ExponentialDecay(
    initial_learning_rate=0.01,  # Tasa de aprendizaje inicial
    decay_steps=1000,           # Número de pasos para aplicar el decay
    decay_rate=0.9,             # Factor de decaimiento
    staircase=False             # Decaimiento continuo
)

# Crear el modelo con Dropout y Regularización L2
mlp_model = Sequential([
    layers.Input(shape=(128*128*3,)),  # Entrada aplanada de las imágenes

    # Primera capa densa con L2 y Dropout
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Apaga el 30% de las neuronas

    # Segunda capa densa con L2 y Dropout
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.3),  # Apaga el 30% de las neuronas

    # Tercera capa densa con L2
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),

    # Capa de salida
    layers.Dense(len(class_names), activation='softmax')  # Número de clases
])

# Compilar el modelo con Decay
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_schedule)

mlp_model.compile(
    optimizer=optimizer,  # Optimización con Decay
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
mlp_model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_40 (Dense)                     │ (None, 256)                 │      12,583,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_41 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_42 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_43 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,624,450 (48.16 MB)

 Trainable params: 12,624,450 (48.16 MB)

 Non-trainable params: 0 (0.00 B)

**- Decay:** El optimizador Adam se configura con una tasa de aprendizaje dinámica. Ésto reduce gradualmente la tasa de aprendizaje a medida que avanza el entrenamiento.

**- Dropout:** Se añade después de las capas densas para apagar aleatoriamente el 30% de las neuronas en cada paso. 

**- Regularización L2:** Añade una penalización a los pesos grandes en las capas densas. 



In [102]:
mlp_model.fit(train_dataset_flat, epochs=10)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 21s 134ms/step - accuracy: 0.4998 - loss: 66.0558
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 20s 135ms/step - accuracy: 0.4976 - loss: 2.6666
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 20s 134ms/step - accuracy: 0.4960 - loss: 2.1387
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 20s 134ms/step - accuracy: 0.5204 - loss: 2.0640
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 20s 135ms/step - accuracy: 0.5067 - loss: 2.1076
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 20s 135ms/step - accuracy: 0.5205 - loss: 1.9537
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 20s 134ms/step - accuracy: 0.5199 - loss: 3.3452
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 20s 135ms/step - accuracy: 0.5335 - loss: 1.4285
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 20s 134ms/step - accuracy: 0.5272 - loss: 1.2149
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 20s 138ms/step - accuracy: 0.5399 - loss: 1.1370


In [103]:
loss, accuracy = mlp_model.evaluate(test_dataset_flat)
print(f"Pérdida del MLP con regularización: {loss}")
print(f"Exactitud del MLP con regularización: {accuracy}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.8584 - loss: 0.9732
Pérdida del MLP con regularización: 0.9919908046722412
Exactitud del MLP con regularización: 0.5405405163764954


### 3.2 Redes convolucionales

In [105]:
train_dir = "data/train"
test_dir = "data/test"

# Carga los datasets
train_dataset = image_dataset_from_directory(
    train_dir,
    labels='inferred',         # Las etiquetas se infieren del nombre de la carpeta
    label_mode='int',          # Etiquetas como enteros
    image_size=(128, 128),     # Redimensiona las imágenes a 128x128
    batch_size=32,             # Tamaño del lote
    shuffle=True               # Mezcla los datos
)

test_dataset = image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    image_size=(128, 128),
    batch_size=32,
    shuffle=False
)
# Inspección de los datos
class_names = train_dataset.class_names
print(f"Clases detectadas: {class_names}")

Found 4733 files belonging to 2 classes.
Found 1184 files belonging to 2 classes.
Clases detectadas: ['chihuahua', 'muffin']


In [106]:

# Opcional: Normalización de imágenes (valores entre 0 y 1)
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))



# Opcional: Configuración para prefetching (mejora el rendimiento durante el entrenamiento)
# AUTOTUNE = tf.data.AUTOTUNE
# train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
# test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


In [107]:
# Definir el modelo de la CNN
model = models.Sequential([
    # Primera capa convolucional
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),

    # Segunda capa convolucional
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Tercera capa convolucional
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Aplanar y agregar capas densas
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),

    # Capa de salida (número de clases)
    layers.Dense(len(class_names), activation='softmax')  # len(class_names) = número de clases
])

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen del modelo
model.summary()

# Entrenar el modelo
model.fit(train_dataset, epochs=10)

# Evaluar el modelo
loss, accuracy = model.evaluate(test_dataset)
print(f"Pérdida: {loss}")
print(f"Exactitud: {accuracy}")


C:\Users\raval\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_44 (Dense)                     │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_45 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_46 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,313,026 (12.64 MB)

 Trainable params: 3,313,026 (12.64 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 25s 158ms/step - accuracy: 0.6370 - loss: 0.6270
Epoch 2/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 160ms/step - accuracy: 0.8542 - loss: 0.3458
Epoch 3/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 159ms/step - accuracy: 0.8663 - loss: 0.3239
Epoch 4/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 158ms/step - accuracy: 0.9101 - loss: 0.2234
Epoch 5/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 158ms/step - accuracy: 0.9346 - loss: 0.1728
Epoch 6/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 160ms/step - accuracy: 0.9525 - loss: 0.1233
Epoch 7/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 158ms/step - accuracy: 0.9587 - loss: 0.1077
Epoch 8/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 24s 158ms/step - accuracy: 0.9646 - loss: 0.0827
Epoch 9/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 25s 169ms/step - accuracy: 0.9786 - loss: 0.0586
Epoch 10/10
148/148 ━━━━━━━━━━━━━━━━━━━━ 26s 175ms/step - accuracy: 0.9891 - loss: 0.0339
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - accuracy: 0.9354 - loss: 0.2404
Pérdida: 0.41183537244796753
Exacti

Para la comparación de ambos modelos:
- Desempeño: ¿Qué modelo tiene mayor precisión?
- Capacidad de generalización: ¿Cuál tiene menor pérdida en el conjunto de prueba?
- Velocidad de entrenamiento: ¿Cuál fue más rápido?